In [1]:
import numpy as np 
import pandas as pd 

In [2]:
data = pd.read_csv("mn_crimes_and_schools.csv")

In [3]:
data.head()

,tax_value,last_sold_price,property_size,zestimate_amount,bathrooms,zestimate_valuation_range_high,tax_year,zestimate_value_change,latitude,zestimate_percentile,...,TOTAL MISDEMEANOR OFFENSES_last_3_years,TOTAL MISDEMEANOR OFFENSES_percent_change,TOTAL NON-SEVEN MAJOR FELONY OFFENSES_last_3_years,TOTAL NON-SEVEN MAJOR FELONY OFFENSES_percent_change,TOTAL SEVEN MAJOR FELONY OFFENSES_last_3_years,TOTAL SEVEN MAJOR FELONY OFFENSES_percent_change,UNAUTHORIZED USE OF A VEHICLE_last_3_years,UNAUTHORIZED USE OF A VEHICLE_percent_change,VEHICLE AND TRAFFIC LAWS_last_3_years,VEHICLE AND TRAFFIC LAWS_percent_change
0,NaN,NaN,NaN,849427.0,1.0,1044795.0,NaN,NaN,40.724448,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
1,1642000.0,NaN,2250.0,NaN,2.0,NaN,2015.0,NaN,40.724420,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
2,1674000.0,NaN,2250.0,7332356.0,1.0,7992268.0,2015.0,NaN,40.724385,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
3,NaN,NaN,NaN,1991289.0,1.0,2708153.0,NaN,NaN,40.724332,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.724845,0.0,...,4072.666667,11.540689,678.666667,0.342633,1364.0,1.539942,5.333333,15.789474,39.333333,-8.256881


In [4]:
data_best_features = pd.read_csv("train_best_features.csv")

In [5]:
data_best_features.columns.values

array(['tax_value', 'zestimate_amount', 'bathrooms', 'latitude',
       'bedrooms', 'home_size',
       'MURDER & NON NEGL. MANSLAUGHTER_last_3_years', 'RAPE_last_3_years',
       'ROBBERY_last_3_years',
       'TOTAL SEVEN MAJOR FELONY OFFENSES_last_3_years',
       'schools_Number Tested', 'schools_mean score',
       'home_type_Apartment', 'home_type_Condominium',
       'home_type_Cooperative', 'home_type_Duplex',
       'home_type_Miscellaneous', 'home_type_MultiFamily2To4',
       'home_type_Quadruplex', 'home_type_SingleFamily',
       'home_type_Townhouse', 'home_type_Unknown',
       'neighborhood_Battery Park', 'neighborhood_Bedford-Stuyvesant',
       'neighborhood_Borough Park', 'neighborhood_Carnegie Hill',
       'neighborhood_Chelsea', 'neighborhood_Chinatown',
       'neighborhood_Clinton', 'neighborhood_East Harlem',
       'neighborhood_East Village', 'neighborhood_Greenwich Village',
       'neighborhood_Hamilton Heights', 'neighborhood_Harlem',
       'neighborhood_

In [6]:
def get_precinct (neighborhood):
    subset = data[data['neighborhood'] == neighborhood]
    precincts = subset['precinct'].unique()
    len_list = []
    for precinct in precincts:
        length = len(subset[subset['precinct'] == precinct])
        len_list += [length]
    max_ind = np.argmax(len_list)
    max_precinct = precincts[max_ind]
    
    return max_precinct

In [7]:
def get_schooldist (neighborhood):
    subset = data[data['neighborhood'] == neighborhood]
    schooldists = subset['schooldistrict'].unique()
    len_list = []
    for schooldist in schooldists:
        length = len(subset[subset['schooldistrict'] == schooldist])
        len_list += [length]
    max_ind = np.argmax(len_list)
    max_schooldist = schooldists[max_ind]
    
    return max_schooldist

In [8]:
def get_feature (neighborhood, home_type, feature):
    neighborhood_new = neighborhood.split("_")[1]
    home_type_new = home_type.split("_")[2]
    subset = data[data['neighborhood'] == neighborhood_new]
    subset = subset[subset['home_type'] == home_type_new]
    subset = subset[np.isfinite(subset[feature])]
    med = np.median(subset[feature])
    
    return med

In [11]:
df = pd.DataFrame(columns=data_best_features.columns.values)

In [12]:
row = data_best_features.loc[1].values

In [13]:
row

array([  2.25000000e+06,   1.99128900e+06,   1.00000000e+00,
         4.07243320e+01,   4.00000000e+00,   1.40000000e+03,
         1.66666667e+00,   1.96666667e+01,   1.27000000e+02,
         1.36400000e+03,   4.58100000e+03,   3.07003400e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00])

In [14]:
df.loc[1] = row

In [15]:
df

,tax_value,zestimate_amount,bathrooms,latitude,bedrooms,home_size,MURDER & NON NEGL. MANSLAUGHTER_last_3_years,RAPE_last_3_years,ROBBERY_last_3_years,TOTAL SEVEN MAJOR FELONY OFFENSES_last_3_years,...,neighborhood_Morningside Heights,neighborhood_North Sutton Area,neighborhood_Soho,neighborhood_Tribeca,neighborhood_Upper East Side,neighborhood_Upper West Side,neighborhood_Washington Heights,neighborhood_West Village,neighborhood_Williamsburg,neighborhood_Yorkville
1,2250000.0,1991289.0,1.0,40.724332,4.0,1400.0,1.666667,19.666667,127.0,1364.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
used_neighborhoods = ['neighborhood_Battery Park', 'neighborhood_Bedford-Stuyvesant',
       'neighborhood_Borough Park', 'neighborhood_Carnegie Hill',
       'neighborhood_Chelsea', 'neighborhood_Chinatown',
       'neighborhood_Clinton', 'neighborhood_East Harlem',
       'neighborhood_East Village', 'neighborhood_Greenwich Village',
       'neighborhood_Hamilton Heights', 'neighborhood_Harlem',
       'neighborhood_Inwood', 'neighborhood_Lower East Side',
       'neighborhood_Morningside Heights',
       'neighborhood_North Sutton Area', 'neighborhood_Soho',
       'neighborhood_Tribeca', 'neighborhood_Upper East Side',
       'neighborhood_Upper West Side', 'neighborhood_Washington Heights',
       'neighborhood_West Village', 'neighborhood_Williamsburg',
       'neighborhood_Yorkville']

In [17]:
used_home_types = ['home_type_Apartment', 'home_type_Condominium',
       'home_type_Cooperative', 'home_type_Duplex',
       'home_type_Miscellaneous', 'home_type_MultiFamily2To4',
       'home_type_Quadruplex', 'home_type_SingleFamily',
       'home_type_Townhouse', 'home_type_Unknown']

In [18]:
num_cols = data_best_features.shape[1]

In [22]:
ind = 0

hometypes_bed_bath = {"home_type_Apartment": [(1,1), (2,1), (3,1)], "home_type_Condominium": [(2, 1.5)], \
             "home_type_Cooperative": [(2,1)], "home_type_Duplex": [(2,2.25)], "home_type_MultiFamily2To4": [(1,1)], \
             "home_type_Quadruplex": [(1,1)], "home_type_SingleFamily": [(2,2)], "home_type_Townhouse": [(4,3)], 
                      "home_type_Miscellaneous": [(1,1)], "home_type_Unknown": [(1,1)]}

for neighborhood in used_neighborhoods:
    for home_type in used_home_types:
        row = [0] * num_cols
        neigh_ind = used_neighborhoods.index(neighborhood)
        home_ind = used_home_types.index(home_type)
        # fill in home type
        row[home_ind + 12] = 1
        # fill in neighborhood
        row[neigh_ind + 22] = 1
        # fill in bathrooms and bedrooms
        beds = hometypes_bed_bath[home_type][0][0]
        baths = hometypes_bed_bath[home_type][0][1]
        row[2] = baths
        row[4] = beds
        # fill in tax value
        tax_val = get_feature(neighborhood, home_type, 'tax_value')
        row[0] = tax_val
        # fill in home size
        home_size = get_feature(neighborhood, home_type, 'home_size')
        row[5] = home_size
        # fill in latitude
        latitude = get_feature(neighborhood, home_type, 'latitude')
        row[3] = latitude
        # fill in crime and school info
        for i in range(6, 12):
            row[i] = get_feature(neighborhood, home_type, data_best_features.columns.values[i])
        
        df.loc[ind] = row
        
        ind += 1
        
        

In [23]:
df.dropna(type=)

,tax_value,zestimate_amount,bathrooms,latitude,bedrooms,home_size,MURDER & NON NEGL. MANSLAUGHTER_last_3_years,RAPE_last_3_years,ROBBERY_last_3_years,TOTAL SEVEN MAJOR FELONY OFFENSES_last_3_years,...,neighborhood_Morningside Heights,neighborhood_North Sutton Area,neighborhood_Soho,neighborhood_Tribeca,neighborhood_Upper East Side,neighborhood_Upper West Side,neighborhood_Washington Heights,neighborhood_West Village,neighborhood_Williamsburg,neighborhood_Yorkville
1,263337.5,0.0,1.50,40.709596,2.0,982.0,0.333333,9.000000,65.666667,1356.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,NaN,0.0,1.00,40.707294,1.0,1700.0,0.333333,9.000000,65.666667,1356.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,0.0,1.00,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,0.0,2.25,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,0.0,1.00,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,380911.0,0.0,1.00,40.716257,1.0,875.0,0.333333,9.000000,65.666667,1356.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NaN,0.0,1.00,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,315456.0,0.0,2.00,40.728474,2.0,1870.0,0.333333,9.000000,65.666667,1356.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NaN,0.0,3.00,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,NaN,0.0,1.00,40.713294,1.0,NaN,0.333333,9.000000,65.666667,1356.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df.to_csv("df_to_predict.csv", index = False)

In [65]:
hometypes_bed_bath = {"home_type_Apartment": [(1,1), (2,1), (3,1)], "home_type_Condominium": [(2, 1.5)], \
             "home_type_Cooperative": [(2,1)], "home_type_Duplex": [(2,2.25)], "home_type_MultiFamily2To4": [(1,1)], \
             "home_type_Quadruplex": [(1,1)], "home_type_SingleFamily": [(2,2)], "home_type_Townhouse": [(4,3)]}

for hometype in hometypes.keys():
    for (bed,bath) in hometypes[hometype]:
        print hometype
        print bed, bath

home_type_Apartment
1 1
home_type_Apartment
2 1
home_type_Apartment
3 1
home_type_Duplex
2 2.25
home_type_Townhouse
4 3
home_type_Cooperative
2 1
home_type_MultiFamily2To4
1 1
home_type_SingleFamily
2 2
home_type_Quadruplex
1 1
home_type_Condominium
2 1.5
